# Modelo de lenguaje con tokenización por caracteres

## Consigna
Seleccionar un corpus de texto sobre el cual entrenar el modelo de lenguaje.

Realizar el pre-procesamiento adecuado para tokenizar el corpus, estructurar el dataset y separar entre datos de entrenamiento y validación.

Proponer arquitecturas de redes neuronales basadas en unidades recurrentes para implementar un modelo de lenguaje.

Con el o los modelos que consideren adecuados, generar nuevas secuencias a partir de secuencias de contexto con las estrategias de greedy search y beam search determístico y estocástico. En este último caso observar el efecto de la temperatura en la generación de secuencias.

## Sugerencias
Durante el entrenamiento, guiarse por el descenso de la perplejidad en los datos de validación para finalizar el entrenamiento. Para ello se provee un callback.

Explorar utilizar SimpleRNN (celda de Elman), LSTM y GRU. rmsprop es el optimizador recomendado para la buena convergencia. No obstante se pueden explorar otros.

In [1]:
import random
import io
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from tensorflow import keras
from tensorflow.keras import layers
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, Dropout
from tensorflow.keras.losses import SparseCategoricalCrossentropy

# descargar de textos.info
import urllib.request

# Para leer y parsear el texto en HTML de wikipedia
import bs4 as bs

En este caso, voy a utilizar el Martin fierro de José Hernandez como corpus

In [2]:
raw_html = urllib.request.urlopen('https://www.textos.info/jose-hernandez/el-gaucho-martin-fierro/ebook')
raw_html = raw_html.read()

# Parsear artículo, 'lxml' es el parser a utilizar
article_html = bs.BeautifulSoup(raw_html, 'lxml')

# Encontrar todos los párrafos del HTML (bajo el tag <p>)
# y tenerlos disponible como lista
article_paragraphs = article_html.find_all('p')

article_text = ''

for para in article_paragraphs:
    article_text += para.text + ' '

# pasar todo el texto a minúscula
article_text = article_text.lower()

In [10]:
# en article text se encuentra el texto de todo el libro
len(article_text)

63273

Separo los caracteres del corpus en un set

In [11]:
chars_vocab = set(article_text)

In [13]:
# la longitud de vocabulario de caracteres es:
len(chars_vocab)

56

In [14]:
# Construimos los dicionarios que asignan índices a caracteres y viceversa.
# El diccionario `char2idx` servirá como tokenizador.
char2idx = {k: v for v,k in enumerate(chars_vocab)}
idx2char = {v: k for k,v in char2idx.items()}

## Tokenizar

In [16]:
# tokenizamos el texto completo
tokenized_text = [char2idx[ch] for ch in article_text]

In [17]:
tokenized_text[:1000]

[46,
 14,
 6,
 54,
 21,
 46,
 31,
 22,
 46,
 33,
 35,
 25,
 47,
 35,
 46,
 14,
 46,
 50,
 14,
 25,
 44,
 14,
 43,
 27,
 13,
 14,
 34,
 46,
 50,
 35,
 31,
 33,
 7,
 2,
 46,
 55,
 22,
 46,
 34,
 14,
 46,
 11,
 5,
 47,
 9,
 22,
 34,
 14,
 48,
 27,
 13,
 6,
 54,
 22,
 46,
 22,
 34,
 46,
 28,
 35,
 31,
 24,
 43,
 22,
 46,
 6,
 54,
 22,
 46,
 34,
 35,
 46,
 55,
 22,
 2,
 11,
 22,
 34,
 14,
 27,
 13,
 54,
 25,
 14,
 46,
 33,
 22,
 25,
 14,
 46,
 22,
 2,
 44,
 43,
 35,
 43,
 55,
 5,
 25,
 14,
 43,
 5,
 14,
 48,
 27,
 13,
 50,
 35,
 31,
 35,
 46,
 34,
 14,
 46,
 14,
 11,
 22,
 46,
 2,
 35,
 34,
 5,
 44,
 14,
 43,
 5,
 14,
 27,
 13,
 50,
 35,
 25,
 46,
 22,
 34,
 46,
 50,
 14,
 25,
 44,
 14,
 43,
 46,
 2,
 22,
 46,
 50,
 35,
 25,
 2,
 54,
 22,
 34,
 14,
 23,
 46,
 33,
 5,
 55,
 35,
 46,
 14,
 46,
 34,
 35,
 2,
 46,
 2,
 14,
 25,
 44,
 35,
 2,
 46,
 55,
 22,
 34,
 46,
 50,
 5,
 22,
 34,
 35,
 27,
 13,
 6,
 54,
 22,
 46,
 14,
 51,
 54,
 55,
 22,
 25,
 46,
 31,
 5,
 46,
 33,
 22,
 25,
 2,
 14,
 31,